In [2]:
'''
!git clone https://github.com/AnnaGhost2713/daia-eon.git
%cd daia-eon/piiranha_refinement
'''

'\n!git clone https://github.com/AnnaGhost2713/daia-eon.git\n%cd daia-eon/piiranha_refinement\n'

In [4]:
import spacy
nlp = spacy.load("de_core_news_md")

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import re
import spacy
import json
from sklearn.metrics import classification_report
import pandas as pd

c:\Users\morit\OneDrive\Uni\02_Master\05_Studium\02_Semester_II\Data Analytics in Applications\VSCode\daia-eon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Label Mapping der einzelnen Identifier

In [3]:
# 📌 Priorität definieren: je höher, desto stärker
PRIORITY_MAP = {
    "regex": 3,
    "piiranha": 1,
    "spacy": 2
}

TARGET_LABELS = ["TITEL", "VORNAME", "NACHNAME", "FIRMA", "TELEFONNUMMER", "EMAIL", "FAX", "STRASSE",
                 "HAUSNUMMER", "POSTLEITZAHL", "WOHNORT", "ZÄHLERNUMMER", "ZÄHLERSTAND", "VERTRAGSNUMMER",
                 "ZAHLUNG", "BANK", "IBAN", "BIC", "DATUM", "GESENDET_MIT", "LINK"]


LABEL_MAP = {
    # spaCy-Labels
    "PER": "NAME", "LOC": "ADRESSE", "ORG": "FIRMA", "DATE": "DATUM", "TIME": "DATUM",
    "GPE": "ADRESSE", "NORP": "GRUPPE", "MONEY": "ZAHLUNG",

    # PIIranha-Labels
    "I-GIVENNAME": "NAME", "I-SURNAME": "NAME", "I-DATEOFBIRTH": "DATUM",
    "I-EMAIL": "KONTAKT", "I-TELEPHONENUM": "KONTAKT", "I-USERNAME": "KONTAKT",
    "I-CREDITCARDNUMBER": "ZAHLUNG",
    "I-ACCOUNTNUM": "VERTRAG", "I-BILLINGNUM": "VERTRAG",
    "I-IDCARDNUM": "VERTRAG", "I-TAXNUM": "VERTRAG",
    "I-CITY": "ADRESSE", "I-ZIPCODE": "ADRESSE", "I-STREET": "ADRESSE", "I-BUILDINGNUM": "ADRESSE",
}

REGEX_PATTERNS = {
    "KONTAKT": r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+|\+49[\d\s\-\(\)]+" ,
    "VERTRAG": r"\b\d{9,10}\b|4\s?0(?:\s?\d){7}",
    "ZAHLUNG": r"\b\d{1,5},\d{2}\s?(€|Euro|Cent)?\b",
    "IBAN": r"DE\d{20}",
    "DATUM": (
        r"\b\d{1,2}\.\d{1,2}\.\d{4}\b|"  # 15.08.2024
        r"\b\d{1,2}\s+(Januar|Februar|März|April|Mai|Juni|Juli|August|"
        r"September|Oktober|November|Dezember)\s+\d{4}\b|"  # 15 August 2024
        r"\b\d{1,2}\.\s+(Januar|Februar|März|April|Mai|Juni|Juli|August|"
        r"September|Oktober|November|Dezember)\s+\d{4}\b|"  # 15. August 2024
        r"\b(Januar|Februar|März|April|Mai|Juni|Juli|August|"
        r"September|Oktober|November|Dezember)\b|"          # August
        r"\b(19|20)\d{2}\b"                                 # Jahreszahlen wie 2023
    )
}


PIIranha Spans

In [4]:
model_name = "iiiorg/piiranha-v1-detect-personal-information"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_piiranha_spans(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, return_offsets_mapping=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    offset_mapping = inputs.pop("offset_mapping")[0].tolist()

    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)[0].tolist()

    spans = []
    current_label = None
    current_start = None

    for i, (start, end) in enumerate(offset_mapping):
        if start == end:  # Special tokens
            continue

        raw_label = model.config.id2label[predictions[i]]
        mapped_label = LABEL_MAP.get(raw_label, None)

        if mapped_label in TARGET_LABELS:
            if current_label == mapped_label:
                continue  # Laufzeit verlängert sich bis label endet
            else:
                # Wenn neuer Start: alten Span abschließen
                if current_label is not None:
                    spans.append({"start": current_start, "end": offset_mapping[i-1][1], "label": current_label})
                current_label = mapped_label
                current_start = start
        else:
            if current_label is not None:
                spans.append({"start": current_start, "end": offset_mapping[i-1][1], "label": current_label})
                current_label = None
                current_start = None

    # Letzten Span abschließen
    if current_label is not None:
        spans.append({"start": current_start, "end": offset_mapping[-1][1], "label": current_label})

    return spans


SpaCy Spans

In [5]:
nlp = spacy.load("../custom_spacy_model_new")

def get_spacy_spans(text):
    doc = nlp(text)
    spans = []
    for ent in doc.ents:
        label = LABEL_MAP.get(ent.label_, ent.label_)
        if label in TARGET_LABELS:
            spans.append({"start": ent.start_char, "end": ent.end_char, "label": label})
    return spans

Regex Spans

In [6]:
def get_regex_spans(text):
    spans = []
    for raw_label, pattern in REGEX_PATTERNS.items():
        mapped_label = LABEL_MAP.get(raw_label, raw_label)  # bleibt bei IBAN = IBAN
        if mapped_label not in TARGET_LABELS:
            continue
        for match in re.finditer(pattern, text):
            spans.append({
                "start": match.start(),
                "end": match.end(),
                "label": mapped_label
            })
    return spans

In [7]:
# Beispieltext zum Testen
sample_text = """
Sehr geehrter Herr John Doe,
Ihre Kundennummer 4012345678 ist aktiv.
Bitte kontaktieren Sie uns unter max@eon.de oder +49 171 1234567.
Ihre Zahlung über 89,99 € wurde am 15. August 2024 verbucht.
"""

# PIIranha-Spans abrufen
piiranha_spans = get_piiranha_spans(sample_text)
spacy_spans = get_spacy_spans(sample_text)
regex_spans = get_regex_spans(sample_text)

# Ergebnisse ausgeben
print(piiranha_spans)
print(spacy_spans)
print(regex_spans)
print("Hi")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[]
[{'start': 20, 'end': 24, 'label': 'VORNAME'}, {'start': 25, 'end': 28, 'label': 'NACHNAME'}, {'start': 48, 'end': 58, 'label': 'VERTRAGSNUMMER'}, {'start': 119, 'end': 135, 'label': 'TELEFONNUMMER'}]
[{'start': 154, 'end': 159, 'label': 'ZAHLUNG'}, {'start': 171, 'end': 186, 'label': 'DATUM'}]
Hi


In [8]:
# 🧠 Duplikate/Überschneidungen optional vereinfachen
def merge_spans(spans):
    return sorted(spans, key=lambda x: x['start'])

def resolve_conflicts(spans):
    # Sortiere Spans nach Startindex, dann nach Länge absteigend (damit äußere zuerst), dann nach Priorität
    spans = sorted(spans, key=lambda x: (x["start"], -(x["end"] - x["start"]), -PRIORITY_MAP.get(x.get("source", ""), 0)))

    resolved = []
    occupied = set()

    for span in spans:
        span_range = set(range(span["start"], span["end"]))
        conflict = False

        for existing in resolved:
            existing_range = set(range(existing["start"], existing["end"]))

            # ❌ Wenn Spans sich überschneiden
            if span_range & existing_range:
                # ➕ Wenn span vollständig in existing liegt oder umgekehrt → Priorität entscheidet
                if span["start"] >= existing["start"] and span["end"] <= existing["end"]:
                    if PRIORITY_MAP[span["source"]] > PRIORITY_MAP[existing["source"]]:
                        resolved.remove(existing)
                        break
                    else:
                        conflict = True
                        break
                elif existing["start"] >= span["start"] and existing["end"] <= span["end"]:
                    if PRIORITY_MAP[span["source"]] < PRIORITY_MAP[existing["source"]]:
                        conflict = True
                        break
                    else:
                        resolved.remove(existing)
                        break
                else:
                    conflict = True
                    break

        if not conflict:
            resolved.append(span)
            occupied.update(span_range)

    return resolved



# 🔐 Redaktion anwenden
def apply_final_redaction(text, spans):
    spans = sorted(spans, key=lambda x: x["start"])
    redacted = []
    last_index = 0

    for span in spans:
        # Text vor dem Span beibehalten
        redacted.append(text[last_index:span["start"]])
        # Ersetzung einfügen
        redacted.append(f"[{span['label']}]")
        # Update der Position
        last_index = span["end"]

    # Rest anhängen
    redacted.append(text[last_index:])
    return ''.join(redacted)


# 🧩 Hauptfunktion
def mask_text_with_all(text):
    all_spans = []

    # Ergebnisse holen und mit 'source' annotieren
    for span in get_regex_spans(text):
        span["source"] = "regex"
        all_spans.append(span)

    for span in get_piiranha_spans(text):
        span["source"] = "piiranha"
        all_spans.append(span)

    for span in get_spacy_spans(text):
        span["source"] = "spacy"
        all_spans.append(span)

    # 🔧 Konflikte auflösen
    spans = resolve_conflicts(all_spans)

    merged = merge_spans(spans)
    return apply_final_redaction(text, merged)

def mask_text_with_single_component(text, component="regex"):
    if component == "regex":
        all_spans = get_regex_spans(text)
    elif component == "piiranha":
        all_spans = get_piiranha_spans(text)
    elif component == "spacy":
        all_spans = get_spacy_spans(text)
    else:
        raise ValueError(f"Unbekannte Komponente: {component}")

    # Optional: Konflikte lösen, falls eine Komponente mehrere Spans mit Überschneidung liefert
    spans = resolve_conflicts(all_spans)
    merged = merge_spans(spans)

    # Gib nur den maskierten Text zurück – analog zur all-Funktion
    return apply_final_redaction(text, merged)


In [15]:
sample = "Vertragsnummer 404149058\n \nFam. Zirme/Rogner\nHenkring 06\n58954 Freising\n \nSehr geehrte Damen und Herren, wir haben am 17.07.23 ein email an euch \ngeschickt und leider nach fast 1,5 Monaten immer noch keine Antwort \nbekommen. Schreiben heute noch ein Mal. \n \nin letzter Abrechnung vom 29.06.2023 ist einen Fehler aufgetreten und zwar \nZählerstand passt nicht. Am 18.06.23 unsere tatsächliche Zählerstand war \n13962 m3. Laut Ihre Abrechnung 14042 m3. Das ist 80m3 Differenz (also 80m3 \nzu viel). Wir bieten Sie alles umzurechnen . Vielen Dank.\n \n \nMfG Fam. Zirme/Rogner\n \n﻿\n"
    
print(get_piiranha_spans(sample))
print(get_spacy_spans(sample))
print(get_regex_spans(sample))
print()
print(sample)
print(mask_text_with_all(sample))
print(mask_text_with_single_component(sample, component="piiranha"))
print(mask_text_with_single_component(sample, component="regex"))
print(mask_text_with_single_component(sample, component="spacy"))

[]
[{'start': 15, 'end': 24, 'label': 'VERTRAGSNUMMER'}, {'start': 27, 'end': 31, 'label': 'VORNAME'}, {'start': 32, 'end': 37, 'label': 'NACHNAME'}, {'start': 38, 'end': 44, 'label': 'NACHNAME'}, {'start': 45, 'end': 53, 'label': 'STRASSE'}, {'start': 54, 'end': 56, 'label': 'HAUSNUMMER'}, {'start': 57, 'end': 62, 'label': 'POSTLEITZAHL'}, {'start': 63, 'end': 71, 'label': 'WOHNORT'}, {'start': 118, 'end': 126, 'label': 'DATUM'}, {'start': 284, 'end': 294, 'label': 'DATUM'}, {'start': 362, 'end': 370, 'label': 'DATUM'}, {'start': 408, 'end': 413, 'label': 'VERTRAGSNUMMER'}, {'start': 439, 'end': 444, 'label': 'VERTRAGSNUMMER'}, {'start': 478, 'end': 482, 'label': 'DATUM'}, {'start': 550, 'end': 554, 'label': 'VORNAME'}, {'start': 555, 'end': 560, 'label': 'NACHNAME'}, {'start': 561, 'end': 567, 'label': 'NACHNAME'}]
[{'start': 284, 'end': 294, 'label': 'DATUM'}]

Vertragsnummer 404149058
 
Fam. Zirme/Rogner
Henkring 06
58954 Freising
 
Sehr geehrte Damen und Herren, wir haben am 17.07

In [12]:
import json
import pandas as pd
from sklearn.metrics import classification_report

# 📂 Testdaten laden
with open(r"C:\Users\morit\OneDrive\Uni\02_Master\05_Studium\02_Semester_II\Data Analytics in Applications\VSCode\daia-eon\data\original\granular_dataset_split_norm_cleaned\test_norm.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

# 🎯 Ground Truth in zeichenbasierte Labels umwandeln
def extract_true_labels(data):
    texts, labels = [], []
    for entry in data:
        text = entry["text"]
        char_labels = ["O"] * len(text)
        for label in entry["labels"]:
            start = label["start"]
            end = label["end"]
            if start < 0 or end > len(text):
                print(f"⚠️ Ungültiger Bereich: {start}-{end} bei Textlänge {len(text)}. Übersprungen.")
                continue
            for i in range(start, end):
                char_labels[i] = label["label"]
        texts.append(text)
        labels.append(char_labels)
    return texts, labels


# 🧩 Hilfsfunktion: PII-Spans → Zeichenbasierte Labels
def spans_to_charlabels(text, spans):
    labels = ["O"] * len(text)
    for span in spans:
        for i in range(span["start"], span["end"]):
            labels[i] = span["label"]
    return labels

# 🔍 Komponenten (diese Funktionen nutzt du aus deinem Erkennungscode)
def run_regex_component(text):
    return spans_to_charlabels(text, get_regex_spans(text))

def run_spacy_component(text):
    return spans_to_charlabels(text, get_spacy_spans(text))

def run_piiranha_component(text):
    return spans_to_charlabels(text, get_piiranha_spans(text))

def run_combined_component(text):
    all_spans = []
    for span in get_regex_spans(text):
        span["source"] = "regex"
        all_spans.append(span)
    for span in get_piiranha_spans(text):
        span["source"] = "piiranha"
        all_spans.append(span)
    for span in get_spacy_spans(text):
        span["source"] = "spacy"
        all_spans.append(span)

    resolved = resolve_conflicts(all_spans)
    merged = merge_spans(resolved)
    return spans_to_charlabels(text, merged)

# 🧪 Evaluation pro Komponente
def evaluate_component(name, component_fn, texts, y_true):
    y_pred = [component_fn(text) for text in texts]
    y_true_flat = [label for seq in y_true for label in seq]
    y_pred_flat = [label for seq in y_pred for label in seq]
    report = classification_report(y_true_flat, y_pred_flat, output_dict=True, zero_division=0)
    df = pd.DataFrame(report).transpose()
    df["component"] = name
    return df

# 🚀 Hauptauswertung starten
texts, y_true = extract_true_labels(test_data)

results = [
    evaluate_component("Regex", run_regex_component, texts, y_true),
    evaluate_component("spaCy", run_spacy_component, texts, y_true),
    evaluate_component("PIIranha", run_piiranha_component, texts, y_true),
    evaluate_component("combined", run_combined_component, texts, y_true)
]

# 📊 Ergebnisse kombinieren
result_df = pd.concat(results).reset_index().rename(columns={"index": "label"})

# 🔍 Relevante PII-Kategorien auswählen
relevant_labels = ["TITEL", "VORNAME", "NACHNAME", "FIRMA", "TELEFONNUMMER", "EMAIL", "FAX", "STRASSE",
                 "HAUSNUMMER", "POSTLEITZAHL", "WOHNORT", "ZÄHLERNUMMER", "ZÄHLERSTAND", "VERTRAGSNUMMER",
                 "ZAHLUNG", "BANK", "IBAN", "BIC", "DATUM", "GESENDET_MIT", "LINK"]
filtered_df = result_df[result_df["label"].isin(relevant_labels)]

# ✅ Finale Übersicht
final_df = filtered_df[["component", "label", "precision", "recall", "f1-score", "support"]]
print(final_df.sort_values(["label", "component"]).round(3))

# 💾 Speichern als CSV
csv_path = "evaluation_results.csv"
final_df.to_csv(csv_path, index=False, encoding="utf-8")

print(f"Ergebnisse gespeichert in '{csv_path}'")



   component         label  precision  recall  f1-score  support
48  PIIranha          BANK      0.000   0.000     0.000     13.0
0      Regex          BANK      0.000   0.000     0.000     13.0
72  combined          BANK      0.000   0.000     0.000     13.0
24     spaCy          BANK      0.000   0.000     0.000     13.0
49  PIIranha         DATUM      0.606   0.103     0.176    194.0
..       ...           ...        ...     ...       ...      ...
43     spaCy  ZÄHLERNUMMER      0.000   0.000     0.000     94.0
68  PIIranha   ZÄHLERSTAND      0.000   0.000     0.000     33.0
20     Regex   ZÄHLERSTAND      0.000   0.000     0.000     33.0
92  combined   ZÄHLERSTAND      0.000   0.000     0.000     33.0
44     spaCy   ZÄHLERSTAND      0.000   0.000     0.000     33.0

[80 rows x 6 columns]
Ergebnisse gespeichert in 'evaluation_results.csv'


In [29]:
text = """Sehr geehrte Damen und Herren\nleider wurde meine Zählernummer immer noch nicht in Ihren Unterlagen geändert. Die Zählernummer lautet bereits seit Vertragsbeginn 1ESY1162808638.\nMit freundlichen Grüßen\nDanica Mohaupt\nD-99763 Erbisdorf\nNettestraße 1\nTel.: +49(0) 993726788\nFax: +49(0)5740 58320\nHandy (02675) 31629\nE-Mail: tomislav80@example.net\n"""

print(len(text))

print(text)

raw_text = "Sehr geehrte Damen und Herren\nleider wurde meine Zählernummer immer noch nicht in Ihren Unterlagen geändert. Die Zählernummer lautet bereits seit Vertragsbeginn 1ESY1162808638.\nMit freundlichen Grüßen\nDanica Mohaupt\nD-99763 Erbisdorf\nNettestraße 1\nTel.: +49(0) 993726788\nFax: +49(0)5740 58320\nHandy (02675) 31629\nE-Mail: tomislav80@example.net\n"
interpreted_text = raw_text.encode().decode('unicode_escape')

print("Länge als Rohtext:", len(raw_text))                # z. B. 344
print("Länge nach Interpretation:", len(interpreted_text))  # z. B. 379


344
Sehr geehrte Damen und Herren
leider wurde meine Zählernummer immer noch nicht in Ihren Unterlagen geändert. Die Zählernummer lautet bereits seit Vertragsbeginn 1ESY1162808638.
Mit freundlichen Grüßen
Danica Mohaupt
D-99763 Erbisdorf
Nettestraße 1
Tel.: +49(0) 993726788
Fax: +49(0)5740 58320
Handy (02675) 31629
E-Mail: tomislav80@example.net

Länge als Rohtext: 344
Länge nach Interpretation: 350
